In [ ]:
from pycalphad import Database, calculate
import numpy as np

# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
all_phases = list(db.phases)
all_phases.sort()

In [10]:
from pycalphad.core.utils import filter_phases, unpack_components

def getPossiblePhases(db, components):
    comps = list(components)
    if 'VA' not in comps:
        comps.append('VA')
    comps = unpack_components(db, comps)
    return filter_phases(db, comps)

T = 2402
components = ['ZR', 'MO', 'W', 'TA', 'V', 'CR', 'CO', 'NI']
amounts = [0.33701280487558255, 0.3128339138729201, 0.09078200772371792, 0.055387487787436254, 0.052765511638791514, 0.05117692034226036, 0.050023289138882085, 0.050018064620409135]
components.append('VA')
possible_phases = getPossiblePhases(db, components)
print(len(possible_phases))

129


In [36]:
results = [np.nan] * len(all_phases)
for i in range(len(all_phases)):
    if all_phases[i] in possible_phases:
        try:
            results[i] = calculate(db, components, all_phases[i] , output = 'GM', P = P, T = T, points = amounts).GM.values.flat[0]
        except:
            continue

In [37]:
np.array(results)[~np.isnan(results)]

array([-5.98218402e+004,             -inf, -5.98218402e+004,
        1.91074569e-204,  6.76293613e+006,  6.76293613e+006,
       -5.98218402e+004, -5.98218402e+004,  6.76293613e+006,
       -2.23950363e+005,  6.36915230e-205,  6.76293613e+006,
       -5.98218402e+004, -2.99109201e+004, -5.98218402e+004,
       -5.98218402e+004, -5.98218402e+004, -5.98218402e+004,
       -5.98218402e+004, -5.98218402e+004, -2.15869300e+005,
       -5.98218402e+004, -2.99109201e+004, -1.94708769e+005,
       -5.98218402e+004, -5.98218402e+004, -5.98218402e+004,
       -2.24919534e+005, -2.22919535e+005, -5.98218402e+004,
       -2.18021011e+005, -5.98218402e+004, -2.24645847e+005,
       -5.98218402e+004, -5.98218402e+004, -5.98218402e+004,
       -5.98218402e+004, -5.98218402e+004, -5.98218402e+004,
       -5.98218402e+004, -5.98218402e+004,  3.18457615e-205,
       -5.98218402e+004,  6.36915230e-205, -5.98218402e+004,
       -5.98218402e+004, -5.98218402e+004, -5.98218402e+004,
       -5.98218402e+004,